Loading the dataset

In [0]:
df = spark.read.table('fordgobike_tripdataa_2017')
df.show()

+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------

Task 1: Open databrics account:
  this is obviously done

Task 2: clean the dataset

In [0]:
df.columns

Out[141]: ['start_time',
 'start time hour',
 'start time minute',
 'start time seconds',
 'end_time',
 'end_time hour',
 'end_time minute',
 'end_time seconds',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'pyment']

In [0]:
# Making help columns that will help us find if there is start time that comes after end time.
df = df.withColumn('start_help', df[1] + df[2]/60 + df[3]/3600)
df = df.withColumn('end_help', df[5] + df[6]/60 + df[7]/3600)

In [0]:
import pyspark.sql.functions as F
df = df.withColumn('test_column', F.when(F.col('start_help') < F.col('end_help'), 0).otherwise(1))

In [0]:
df.groupBy('test_column').count().show() # this shows us that there are 10829 rows that have start date older than end date.

+-----------+------+
|test_column| count|
+-----------+------+
|          1| 10829|
|          0|508871|
+-----------+------+



In [0]:
df = df.withColumn('new_start', F.when(F.col('test_column') == 1, df[4]).otherwise(df[0]))   # making new columns that will serve as source to update our time columns
df = df.withColumn('new_start_hour', F.when(F.col('test_column') == 1, df[5]).otherwise(df[1]))
df = df.withColumn('new_start_minute', F.when(F.col('test_column') == 1, df[6]).otherwise(df[2]))
df = df.withColumn('new_start_seconds', F.when(F.col('test_column') == 1, df[7]).otherwise(df[3]))
df = df.withColumn('new_end', F.when(F.col('test_column') == 1, df[0]).otherwise(df[4]))
df = df.withColumn('new_end_hour', F.when(F.col('test_column') == 1, df[1]).otherwise(df[5]))
df = df.withColumn('new_end_minute', F.when(F.col('test_column') == 1, df[2]).otherwise(df[6]))
df = df.withColumn('new_end_seconds', F.when(F.col('test_column') == 1, df[3]).otherwise(df[7]))

In [0]:
df = df.withColumn('start_time', df.new_start)  #updating columns with fixed values
df = df.withColumn('start time hour', df.new_start_hour)
df = df.withColumn('start time minute', df.new_start_minute)
df = df.withColumn('start time seconds', df.new_start_seconds)
df = df.withColumn('end_time', df.new_end)
df = df.withColumn('end_time hour', df.new_end_hour)
df = df.withColumn('end_time minute', df.new_end_minute)
df = df.withColumn('end_time seconds', df.new_end_seconds)

In [0]:
df = df.withColumn('start_help_1', df[1] + df[2]/60 + df[3]/3600)  # making the same test as the one at the beginning to see if we fixed the issue
df = df.withColumn('end_help_1', df[5] + df[6]/60 + df[7]/3600)

In [0]:
df = df.withColumn('test_column_1', F.when(F.col('start_help_1') < F.col('end_help_1'), 0).otherwise(1))

In [0]:
df.select('test_column_1').distinct().count() # here we can see there is only 1 unique value

Out[149]: 1

In [0]:
df.groupBy('test_column_1').count().show() # here we can see that 1 unique value is "0" and it fills entire column, which means issue is solved.

+-------------+------+
|test_column_1| count|
+-------------+------+
|            0|519700|
+-------------+------+



In [0]:
cols_to_drop = [ 'start_help',
 'end_help',
 'test_column',
 'new_start',
 'new_start_hour',
 'new_start_minute',
 'new_start_seconds',
 'new_end',
 'new_end_hour',
 'new_end_minute',
 'new_end_seconds',
 'start_help_1',
 'end_help_1',
 'test_column_1']

In [0]:
df = df.drop(*cols_to_drop)  # just dropping columns we no longer need and the task is done.

Task 2: Calculate distance of each trip using haversine library and add the result to the dataset

In [0]:
from haversine import haversine

In [0]:
# df_test = df_test.withColumn('loc1', F.concat_ws(', ', F.col('start_station_latitude'), F.col('start_station_longitude')))
# df_test = df_test.withColumn('loc2', F.concat_ws(', ', F.col('end_station_latitude'), F.col('end_station_longitude')))

In [0]:
def haversine_f(lat1, lon1, lat2, lon2):
    return haversine( (lat1, lon1), (lat2, lon2) )

In [0]:
haversine_udf = udf(haversine_f)

In [0]:
df = df.withColumn('haversine_distance', haversine_udf('start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude'))

In [0]:
df.show(5)

+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|haversine_distance|
+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+-